In [ ]:
# Until v4.26.0 is out, requires:
# pip install --upgrade git+https://github.com/huggingface/transformers/

import random
import time

import diffusers
import torch
import PIL

# To test performance delta.
use_cuda = torch.cuda.is_available()

def inference(prompt, n_images, guidance, steps, seed, img, neg_prompt):
    generator = None
    if use_cuda:
        generator = torch.Generator("cuda").manual_seed(seed) if seed else None
    elif seed:      
        generator = torch.Generator()
        generator.manual_seed(seed)
    pipe = diffusers.StableDiffusionDepth2ImgPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-depth",
        revision="fp16" if use_cuda else "fp32",
        torch_dtype=torch.float16 if use_cuda else torch.float32)
    pipe.scheduler = diffusers.DPMSolverMultistepScheduler.from_config(
        pipe.scheduler.config)
    if use_cuda:
        pipe.to("cuda")
        pipe.enable_attention_slicing()
    return pipe(
        prompt,
        num_images_per_prompt=n_images,
        negative_prompt=neg_prompt,
        image=img,
        num_inference_steps=steps,
        guidance_scale=guidance,
        # width=width,
        # height=height,
        generator=generator).images

def run(prompt, image):
    n_images = 1 # number of images
    neg_prompt = ""
    guidance = 7.5 # max = 15
    steps = 25 # [2, 100]
    seed = 10 # random
    # seed = random.randint(0, 2147483647)
    print(f"Seed: {seed}")
    start = time.time()
    gallery = inference(prompt, n_images, guidance, steps, seed, image, neg_prompt)
    print("Took %.1fs" % (time.time()-start))
    return gallery[0]

img = run("war zone", PIL.Image.open("out/bench.png"))
img.save("out/war_zone50.png")
img